In [ ]:
import numpy as np
import qililab as ql

from utils.qst_qpt_helper_functions import *

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

import os

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit


api = API(ConnectionConfiguration(username="qat", api_key="meow"))


api.select_device_id(9)

Is setting the logger perhaps a bit ugly for remote users to have to do manually? I have not seen any experiment where it is preferred to not be set to a higher level

In [ ]:
ql.logger.setLevel(40)  # Set qililab's logger to a higher level so it only shows error messages

In [ ]:
# Amplitude set in Qblox terms (0.5 times mV)
Time_START = 0
Time_STOP = 200000
Time_STEP = 2000

SQUARE_duration = 4000
GAUSS_duration = 2000

NSHOTS = 2000

In [ ]:
# The shape of the pulses is introduced here
gauss_wf = ql.Gaussian(amplitude=0.5, duration=GAUSS_duration, num_sigmas=4.0)  # Drag pulse executing pi/2 rotation
square_wf = ql.Square(amplitude=1, duration=SQUARE_duration)  # square wavefront (readout pulse)
weights_shape = ql.Square(amplitude=1, duration=SQUARE_duration)  # This shape implies that the weights are uniform

# Clarify when we need to add sync()

In [ ]:
# Building the Qprogram
qp_t2 = ql.QProgram()

# Defining the variable Frequency for the driving pulse
wait_time = qp_t2.variable(ql.Domain.Time)  # Possible domains: Scalar, Time, Frequency, Phase and Voltage

# Averaging over NSHOTS iterations
with qp_t2.average(NSHOTS):
    # Loop over all wait times
    with qp_t2.for_loop(variable=wait_time, start=Time_START, stop=Time_STOP, step=Time_STEP):
        qp_t2.play(bus="drive_q0_bus", waveform=gauss_wf)  # execute the pi pulse
        qp_t2.wait(bus="drive_q0_bus", duration=wait_time)  # wait

        qp_t2.sync()  # so that the next bus (for the readout) executes after the previous (drive)

        # READOUT PULSE
        qp_t2.play(bus="readout_q0_bus", waveform=square_wf, wait_time=4)  ## execute readout pulse
        qp_t2.acquire(
            bus="readout_q0_bus", weights=ql.IQPair(I=weights_shape, Q=weights_shape)
        )  # Collect data from readout

full_qprogram = qp_t2.to_dict()

In [ ]:
Wait_duration_list = np.arange(Time_START, Time_STOP, Time_STEP)

In [ ]:
result_id = api.execute(full_qprogram)[0]
print(result_id)

In [ ]:
results = api.get_result(result_id)
data_probabilities = process_returned_dataformat(results, nqubits=1)

In [ ]:
def ramsey_fit(xdata: np.ndarray, amplitude: float, rate: float, offset: float, frequency: float, phase: float) -> np.ndarray:  # type: ignore  # pylint: disable=arguments-differ
    """Exponential model function.

    It must take the independent variable as the first argument and the parameters to fit as separate remaining
    arguments.

    Args:
        xdata (ndarray): data of the x-axis
        amplitude (float): amplitude of the exponential function
        rate (float): rate of growth or decay
        frequency (float): frequency of Ramsey oscillations
        phase (float): phase offset of the cosine. Should be 0 for the gs T2 and np.pi/2 for the 1exc.
        offset (float): offset. Sould be 0.5.
    """
    return amplitude * np.exp(rate * xdata) * np.cos(xdata * frequency + phase) + offset


def ramsey_fit_gs_simple(xdata: np.ndarray, rate: float, frequency: float) -> np.ndarray:  # type: ignore  # pylint: disable=arguments-differ
    """Exponential model function.

    It must take the independent variable as the first argument and the parameters to fit as separate remaining
    arguments.

    Args:
        xdata (ndarray): data of the x-axis
        rate (float): rate of growth or decay
        frequency (float): frequency of Ramsey oscillations
    """
    return 1 * np.exp(rate * xdata) * np.cos(xdata * frequency) + 0.5

## Not finished yet

In [ ]:
probabilities_gs = data_probabilities[:, 0]


fit_up2 = -1
coeffs_fit = np.polyfit(Wait_duration_list[:fit_up2], np.log10(probabilities_gs[:fit_up2]), 1)
poly1d_fn = np.poly1d(coeffs_fit)  # poly1d_fn is now a function which takes in x and returns an estimate for y
exponent = coeffs_fit[0]
T2 = -1 / exponent

# Plot the data and the fitted function
plt.figure(figsize=(8, 6))
plt.plot(Wait_duration_list, np.log10(probabilities_gs), label="data")
plt.plot(Wait_duration_list, poly1d_fn(Wait_duration_list), "r", label=f"Fitted T1={T2/1000:.3f} us")
plt.legend()
plt.xlabel("Wait time (ns)")
plt.ylabel("Prob($|1\\rangle$)")